In [1]:
import pandas as pd
from pymongo import MongoClient

data_collection = MongoClient("mongodb://localhost:27017")["ISRO"]["data_collection_v4"]

In [2]:
df = pd.read_csv("/home/sm/Public/Inter-IIT/Astral-Ray-Scratchpad/Soumik/data/isro/elemental_abundances_by_isro.csv")
df.head()

,V0_LATITUDE,V0_LONGITUDE,V1_LATITUDE,V1_LONGITUDE,V2_LATITUDE,V2_LONGITUDE,V3_LATITUDE,V3_LONGITUDE,MG_WT,MG_FIT_UNC,...,AL_SYS_UNC,AL_TOT_UNC,SI_WT,SI_FIT_UNC,SI_SYS_UNC,SI_TOT_UNC,FE_WT,FE_FIT_UNC,FE_SYS_UNC,FE_TOT_UNC
0,-44.76,120.40,-50.51,120.50,-50.49,121.91,-44.74,121.66,4.09,0.20,...,3.63,3.67,18.02,0.53,4.38,4.41,3.64,0.0,4.1,4.1
1,-54.45,120.57,-60.20,120.72,-60.17,122.53,-54.43,122.12,3.71,0.35,...,3.63,3.68,17.80,0.66,4.38,4.43,3.04,0.0,4.1,4.1
2,-59.29,120.69,-65.04,120.88,-65.01,123.02,-59.27,122.45,4.81,0.58,...,3.63,3.73,17.70,1.02,4.38,4.49,3.13,0.0,4.1,4.1
3,49.72,116.61,44.01,116.85,44.02,117.99,49.74,117.88,5.09,1.62,...,3.63,4.27,18.92,2.76,4.38,5.18,3.24,0.0,4.1,4.1
4,44.83,116.82,39.11,117.01,39.13,118.07,44.85,117.97,6.40,1.59,...,3.63,4.18,17.12,2.61,4.38,5.09,3.63,0.0,4.1,4.1


In [3]:
import datetime


with open("points_v3.csv", "w") as f:
    for idx, row in df.iterrows():
        v0_lat = row[0]
        v0_lon = row[1]
        v1_lat = row[2]
        v1_lon = row[3]
        v2_lat = row[4]
        v2_lon = row[5]
        v3_lat = row[6]
        v3_lon = row[7]

        mg_wt = row[8]
        al_wt = row[12]
        si_wt = row[16]
        fe_wt = row[20]

        centroid_lat = (v0_lat + v1_lat + v2_lat + v3_lat) / 4
        centroid_lon = (v0_lon + v1_lon + v2_lon + v2_lon) / 4

        # top_mid_lat_1 = (v0_lat + v3_lat) / 2
        # top_mid_lon_1 = (v0_lon + v3_lon) / 2

        left_mid_lat_1 = (v0_lat + 2 * v1_lat) / 3
        left_mid_lon_1 = (v0_lon + 2 * v1_lon) / 3

        left_mid_lat_2 = (2 * v0_lat + v1_lat) / 3
        left_mid_lon_2 = (2 * v0_lon + v1_lon) / 3

        # bottom_mid_lat_1 = (v1_lat + v2_lat) / 2
        # bottom_mid_lon_1 = (v1_lon + v2_lon) / 2

        right_mid_lat_1 = (v2_lat + 2 * v3_lat) / 3
        right_mid_lon_1 = (v2_lon + 2 * v2_lon) / 3

        right_mid_lat_2 = (2 * v2_lat + v3_lat) / 3
        right_mid_lon_2 = (2 * v2_lon + v3_lon) / 3

        wt_str = f"{mg_wt},{al_wt},{si_wt},{fe_wt}"
        wt_dict = {
            "mg": mg_wt,
            "al": al_wt,
            "si": si_wt,
            "fe": fe_wt,
        }

        lat_lon_tuples = [
            (v0_lat, v0_lon),
            (v1_lat, v1_lon),
            (v2_lat, v2_lon),
            (v3_lat, v3_lon),
            (centroid_lat, centroid_lon),
            (left_mid_lat_1, left_mid_lon_1),
            (left_mid_lat_2, left_mid_lon_2),
            (right_mid_lat_1, right_mid_lon_1),
            (right_mid_lat_2, right_mid_lon_2),
        ]

        for lat, lon in lat_lon_tuples:
            f.write(f"{lat:.2f},{lon:.2f},{lat:.2f}_{lon:.2f}.fits,{wt_str}")
            data_collection.insert_one(
                {
                    "latitude": f"{lat:.2f}",
                    "longitude": f"{lon:.2f}",
                    "filepath": f"{lat:.2f}_{lon:.2f}.fits",
                    "true_wt": wt_dict,
                    "status": False,
                    "last_served": datetime.datetime.fromtimestamp(0),
                }
            )

/tmp/ipykernel_1132454/1531298336.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  v0_lat = row[0]
/tmp/ipykernel_1132454/1531298336.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  v0_lon = row[1]
/tmp/ipykernel_1132454/1531298336.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  v1_lat = row[2]
/tmp/ipykernel_1132454/1531298336.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future ve